# ED VISISTS - jkc

## Ingest

### Libraries

In [ ]:
import pandas as pd
from pandas import to_datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from fbprophet import Prophet 

In [ ]:
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(22, 8)})

### Load Data

In [ ]:
filename = 'Y:\Data\ED_visits_2015_2020.csv'
edvisit = pd.read_csv(filename, parse_dates=['ed_door_time', 'roomed_time', 'ed_discharge_time', \
                                             'triage_time', 'sort_time', 'notify_time', 'refer_time', \
                                             'adt_arrival_time', 'hosp_admsn_time', 'hosp_disch_time', \
                                            'record_create_time'], index_col='record_create_time')
edvisit_colName = list(edvisit)

In [ ]:
#edvisit.dtypes
edvisit.info()

In [ ]:
edvisit.head(5)

In [ ]:
# Nulls
#print(edvisit.isnull().sum())
null_columns=edvisit.columns[edvisit.isnull().any()]

In [ ]:
edvisit.index

In [ ]:
print('Min: ', edvisit.index.date.min())
print('Max: ', edvisit.index.date.max())

In [ ]:
# Remove the 2014 records
edvisit = edvisit.loc['2015':'2020']

In [ ]:
edvisit.head(3)

## Feature Engineering

In [ ]:
# Make acuity_level an integer
#print(edvisit['acuity_level'].unique())
acuity_level_dict = {'L1':1, 'L2':2, 'L3':3, 'L4':4, 'L5':5, 'PVO':0}
edvisit['acuity_level_int'] = edvisit['acuity_level'].map(acuity_level_dict)
#edvisit.head(3)

In [ ]:
# Make DISPOSITION an integer
#print(edvisit['DISPOSITION'].unique())
DISPOSITION_list = edvisit['DISPOSITION'].unique()
DISPOSITION_dict = dict(zip(DISPOSITION_list, range(len(DISPOSITION_list))))
edvisit['DISPOSITION_int'] = edvisit['DISPOSITION'].map(DISPOSITION_dict)
#edvisit.head(3)

In [ ]:
# Make admit_conf_stat an integer
print(edvisit['admit_conf_stat'].unique())
# Note - nothing other than Confirmed, so drop column
edvisit = edvisit.drop(columns=['admit_conf_stat'])

In [ ]:
# Create time attributes
edvisit['Date'] = edvisit.index.date
edvisit['Year'] = edvisit.index.year
edvisit['Month'] = edvisit.index.month
edvisit['Day'] = edvisit.index.day
edvisit['DayofWeek'] = edvisit.index.weekday
edvisit['DayofYear'] = edvisit.index.dayofyear
edvisit.tail(3)

In [ ]:
# Create door2dis
#edvisit['ed_discharge_time'].isnull().sum()                              # note 1427 NULLS
#edvisit.loc[edvisit['ed_discharge_time'].isnull() != True]
edvisit['door2dis'] = edvisit['ed_discharge_time'] - edvisit['ed_door_time']

## EDA

In [ ]:
edvisit.describe()

In [ ]:
# Records created per day
recsperday = edvisit.groupby(['Date']).size()

sns.lineplot(data=recsperday)
# 7 day rolling median
sns.lineplot(data=recsperday.rolling(window=7).median())

In [ ]:
# Acuity levels combined (same as record create minus the 2k NaN's)
edvisit.groupby(['Date'])['acuity_level'].count()

In [ ]:
# Acuity levels separated
edvisit.groupby(['Date', 'acuity_level'])['Date'].count().unstack().plot()

In [ ]:
# Covid zoom-in by slicing
covidTransition = edvisit.loc['2019-10':'2021']

In [ ]:
covidTransition.groupby(['Date', 'acuity_level'])['Date'].count().unstack().plot()

## Extract data for aggregation/EDA and modeling

In [ ]:
# Extract acuity levels's  (need to loop/function this)
edviL1 = edvisit[edvisit['acuity_level'] == 'L1']
list_edviL1 = edviL1['acuity_level'].resample('D').count()

edviL2 = edvisit[edvisit['acuity_level'] == 'L2']
list_edviL2 = edviL2['acuity_level'].resample('D').count()

edviL3 = edvisit[edvisit['acuity_level'] == 'L3']
list_edviL3 = edviL3['acuity_level'].resample('D').count()

edviL4 = edvisit[edvisit['acuity_level'] == 'L4']
list_edviL4 = edviL4['acuity_level'].resample('D').count()

edviL5 = edvisit[edvisit['acuity_level'] == 'L5']
list_edviL5 = edviL5['acuity_level'].resample('D').count()

edvipvo = edvisit[edvisit['acuity_level'] == 'PVO']
list_edvipvo = edvipvo['acuity_level'].resample('D').count()

In [ ]:
# Create dataframe
edvi_BY_DAY = pd.DataFrame()
edvi_BY_DAY['L5'] = list_edviL5
edvi_BY_DAY['L4'] = list_edviL4
edvi_BY_DAY['L3'] = list_edviL3
edvi_BY_DAY['L2'] = list_edviL2
edvi_BY_DAY['L1'] = list_edviL1
edvi_BY_DAY['PVO'] = list_edvipvo
edvi_BY_DAY['acTot'] = edvi_BY_DAY['L5'] + edvi_BY_DAY['L4'] + edvi_BY_DAY['L3'] + edvi_BY_DAY['L2'] + edvi_BY_DAY['L1'] #+ edvi_BY_DAY['PVO']
edvi_BY_DAY['highAcuity'] = edvi_BY_DAY['L3'] + edvi_BY_DAY['L2'] + edvi_BY_DAY['L1']
edvi_BY_DAY['lowAcuity'] = edvi_BY_DAY['L5'] + edvi_BY_DAY['L4'] #+ edvi_BY_DAY['PVO']
#
#      NOTE - ABOVE PVO ommitted from series dues to NULLS?????
#

# Converting the index as date
#edvi_BY_DAY.index = edvi_BY_DAY.to_datetime(edvi_BY_DAY.index)
edvi_BY_DAY = edvi_BY_DAY.reset_index()
edvi_BY_DAY = edvi_BY_DAY.set_index('record_create_time')
edvi_BY_DAY.sort_index(inplace=True)

# Create time attributes
edvi_BY_DAY['Date'] = edvi_BY_DAY.index.date
edvi_BY_DAY['Year'] = edvi_BY_DAY.index.year
edvi_BY_DAY['Month'] = edvi_BY_DAY.index.month
edvi_BY_DAY['Day'] = edvi_BY_DAY.index.day
edvi_BY_DAY['WeekdayName'] = edvi_BY_DAY.index.weekday
edvi_BY_DAY['DayofWeek'] = edvi_BY_DAY.index.weekday
edvi_BY_DAY['DayofYear'] = edvi_BY_DAY.index.dayofyear

In [ ]:
cols_plot = ['highAcuity', 'lowAcuity','acTot', 'L1', 'L2', 'L3', 'L4', 'L5', 'PVO']
axes = edvi_BY_DAY[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(22, 22), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (Acuity)')

## Seasonality

In [ ]:
fig, axes = plt.subplots(9, 1, figsize=(22, 22), sharex=True)
for name, ax in zip(['highAcuity', 'lowAcuity', 'acTot', 'L1', 'L2', 'L3', 'L4', 'L5', 'PVO'], axes):
    sns.boxplot(data=edvi_BY_DAY, x='Month', y=name, ax=ax)
ax.set_ylabel('Daily Totals (acuity)')
ax.set_title(name)
# Remove the automatic x-axis label from all but the bottom subplot
#if ax != axes[-1]:
#    ax.set_xlabel('')

## Weekly

In [ ]:
sns.boxplot(data=edvi_BY_DAY, x='WeekdayName', y='acTot');

In [ ]:
# Resampling to weekly for visualization
data_columns = ['highAcuity', 'lowAcuity', 'acTot', 'L1', 'L2', 'L3', 'L4', 'L5', 'PVO']
# Resample to weekly frequency, aggregating with mean
edvi_BY_WEEK_mean = edvi_BY_DAY[data_columns].resample('W').mean()
#edvi_BY_WEEK_mean.head(3)

In [ ]:
# Start and end of the date range to extract
start, end = '2015', '2021'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY.loc[start:end, 'acTot'], linestyle='-', linewidth=0.5, label='Daily')
ax.plot(edvi_BY_WEEK_mean.loc[start:end, 'acTot'], markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.set_ylabel('acTot')
ax.legend();

## Rolling windows

In [ ]:
# Compute the centered 7-day rolling mean
edvi_BY_DAY_7d = edvi_BY_DAY[data_columns].rolling(7, center=True).mean()
#edvi_BY_DAY_7d.head(5)

In [ ]:
# Start and end of the date range to extract
start, end = '2019', '2020'
# Plot daily, weekly resampled, and 7-day rolling mean time series together
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY.loc[start:end, 'acTot'], linestyle='-', linewidth=0.5, label='Daily')
ax.plot(edvi_BY_WEEK_mean.loc[start:end, 'acTot'], markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.plot(edvi_BY_DAY_7d.loc[start:end, 'acTot'], linestyle='-', label='7-d Rolling Mean')
ax.set_ylabel('acTot')
ax.legend();

## Trends

In [ ]:
edvi_BY_DAY_365d = edvi_BY_DAY[data_columns].rolling(window=365, center=True, min_periods=360).mean()

In [ ]:
# Plot daily, 7-day rolling mean, and 365-day rolling mean time series
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY['acTot'], marker='.', markersize=2, color='0.25', linestyle='None', label='Daily')
ax.plot(edvi_BY_DAY_7d['acTot'], linewidth=2, label='7-d Rolling Mean')
ax.plot(edvi_BY_DAY_365d['acTot'], color='0.2', linewidth=3,
label='Trend (365-d Rolling Mean)')
# Set x-ticks to yearly interval and add legend and labels
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('acTot')
ax.set_title('Trends in EDVISITS');

In [ ]:
# Start and end of the date range to extract
start, end = '2015', '2021'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY_7d.loc[start:end,'highAcuity'], linewidth=2, label='highAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'lowAcuity'], linewidth=2, label='lowAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'acTot'], linewidth=2, label='acTot 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L1'], linewidth=2, label='L1 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L2'], linewidth=2, label='L2 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L3'], linewidth=2, label='L3 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L4'], linewidth=2, label='L4 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L5'], linewidth=2, label='L5 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'PVO'], linewidth=2, label='PVO 7-d Rolling Mean')
ax.set_ylabel('acTot')
ax.legend();

## Zoom in to se COVID drop

In [ ]:
# Start and end of the date range to extract
start, end = '2020-1', '2020-6'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY_7d.loc[start:end,'highAcuity'], linewidth=2, label='highAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'lowAcuity'], linewidth=2, label='lowAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'acTot'], linewidth=2, label='acTot 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L1'], linewidth=2, label='L1 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L2'], linewidth=2, label='L2 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L3'], linewidth=2, label='L3 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L4'], linewidth=2, label='L4 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L5'], linewidth=2, label='L5 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'PVO'], linewidth=2, label='PVO 7-d Rolling Mean')
ax.set_ylabel('acTot')
ax.legend();

## Even more

In [ ]:
# Start and end of the date range to extract
start, end = '2020-3-1', '2020-4-15'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(edvi_BY_DAY_7d.loc[start:end,'highAcuity'], linewidth=2, label='highAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'lowAcuity'], linewidth=2, label='lowAcuity 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'acTot'], linewidth=2, label='acTot 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L1'], linewidth=2, label='L1 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L2'], linewidth=2, label='L2 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L3'], linewidth=2, label='L3 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L4'], linewidth=2, label='L4 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'L5'], linewidth=2, label='L5 7-d Rolling Mean')
ax.plot(edvi_BY_DAY_7d.loc[start:end,'PVO'], linewidth=2, label='PVO 7-d Rolling Mean')
ax.set_ylabel('acTot')
ax.legend();

## Prophet Forecasting - PREDICTING Q4

In [ ]:
%matplotlib inline 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [ ]:
# Remove the 2020 Q4 records
edvi_BY_DAY_Q4PRED = edvi_BY_DAY.loc['2015':'2020-9']

In [ ]:
edvi_BY_DAY_Q4PRED.head()

In [ ]:
edvi_BY_DAY_Q4PRED.tail()

In [ ]:
df = edvi_BY_DAY_Q4PRED[['acTot']]
df = df.reset_index()
df = df.rename(columns={"record_create_time": "ds", "acTot": "y"})
df['ds']= to_datetime(df['ds'])
df
#df.sort_index(inplace=True)

### Experiment with holidays

In [ ]:
bronco_homegames = pd.DataFrame({
    'holiday' : 'broncos_prs_home',
    'ds' : pd.to_datetime(['08-29-2015', '09-03-2015', '09-13-2015', '10-04-2015', 
                           '11-01-2015', '11-15-2015', '11-29-2015', '12-13-2015',
                           '12-28-2015', '01-03-2016', '08-20-2016', '08-27-2016',
                           '09-08-2016', '09-18-2016', '10-09-2016', '10-24-2016',
                           '10-30-2016', '11-27-2016', '12-18-2016', '01-01-2017',
                           '08-26-2017', '08-31-2017', '09-11-2016', '09-17-2017',
                           '10-01-2017', '10-15-2017', '11-12-2017', '11-19-2017',
                           '12-10-2017', '12-31-2017', '08-11-2018', '08-18-2018',
                           '09-09-2018', '09-16-2018', '10-01-2018', '10-14-2018',
                           '11-04-2018', '11-25-2018', '12-15-2018', '12-30-2018',
                           '08-19-2019', '08-29-2019', '09-15-2019', '09-29-2019',
                           '10-13-2019', '10-17-2019', '11-03-2019', '12-01-2019',
                           '12-22-2019', '12-29-2019', '09-14-2020', '09-27-2020',
                           '10-25-2020', '11-01-2020', '11-22-2020', '11-29-2020',
                           '12-19-2020', '01-03-2021'])})
friday_13th = pd.DataFrame({
    'holiday' : 'friday_13th',
    'ds' : pd.to_datetime(['02-13-2015', '03-13-2015', '11-13-2015', '05-13-2016',
                           '01-13-2017', '10-13-2017', '04-13-2018', '07-13-2018',
                           '09-13-2019', '12-13-2019', '03-13-2020', '11-13-2020'])})
bronco_playoffs = pd.DataFrame({
    'holiday' : 'broncos_playoffs',
    'ds' : pd.to_datetime(['01-17-2016', '01-24-2016'])})
bronco_superbowl = pd.DataFrame({    
    'holiday' : 'broncos_superbowl_win',
    'ds' : pd.to_datetime(['02-07-2016'])})
holidays = pd.concat((bronco_homegames, bronco_playoffs, bronco_superbowl, friday_13th))

### Create regressor for covid behavior

In [ ]:
# Create regressor to account for preCovid and postCovid times
# 100 down to 50 up to 75, can be fine tuned
covidRegressor = pd.read_csv(r'C:\Users\173566\Desktop\Reg2_scaled100.csv')
df['covidRegressor']= covidRegressor['regressor']

### Model with Prophet

In [ ]:
model = Prophet(changepoint_range=0.95, changepoint_prior_scale=0.25, daily_seasonality=True, 
                seasonality_mode='multiplicative')#, holidays=holidays)
model.add_country_holidays(country_name='US')
model.add_regressor('covidRegressor')
model.fit(df)

In [ ]:
# Make future dates
future_data = model.make_future_dataframe(freq='d', periods=92)
future_data['covidRegressor']= covidRegressor['regressor']
future_data.tail()

In [ ]:
# Make forecast
forecast = model.predict(future_data)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
# Forecast components
fig2 = model.plot_components(forecast)

In [ ]:
# Trend changepoints
changepointsList = model.changepoints
model.changepoints

In [ ]:
# Trend changepoints
figure = model.plot(forecast)
for changepoint in model.changepoints:
    plt.axvline(changepoint,ls='--', lw=1)

In [ ]:
# Changepoint magnitude
deltas = model.params['delta'].mean(0)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111)
ax.bar(range(len(deltas)), deltas)
ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
ax.set_ylabel('Rate change')
ax.set_xlabel('Potential changepoint')
fig.tight_layout()

## Cross validation

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='1095 days',
                         period='365 days', horizon = '720 days')

In [ ]:
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

df_p = performance_metrics(df_cv)
df_p.head()

## Metrics for Submission

In [ ]:
# Make forecast
Q4_forecast = forecast[['ds', 'yhat']].set_index('ds')
Q4_forecast = Q4_forecast.loc['2020-10':'2020-12']
Q4_forecast['yhat'] = round(Q4_forecast['yhat'])

In [ ]:
Q4_actual = edvi_BY_DAY[['acTot']].loc['2020-10':'2020-12']
Q4_actual = Q4_actual.rename_axis('ds')

In [ ]:
Q4Metrics = Q4_forecast.merge(Q4_actual, on='ds')
Q4Metrics.head()

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Compute the root-mean-square
Q4Metrics_rmse = np.sqrt(mean_squared_error(Q4Metrics.yhat, Q4_actual.acTot))
print(Q4Metrics_rmse)

In [ ]:
Q4Metrics_mae = mean_absolute_error(Q4Metrics.yhat, Q4_actual.acTot)
print(Q4Metrics_mae)

In [ ]:
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_Predicted - Y_actual)/Y_actual))*100
    return mape
Q4Metrics_mape = MAPE(Q4Metrics.yhat, Q4_actual.acTot)
print(Q4Metrics_mape)

In [ ]:
#mean absolute error percenatage
np.sum(np.abs(Q4Metrics.yhat - Q4_actual.acTot)) / np.sum(Q4_actual.acTot)

In [ ]:
Q4Metrics['Delta'] = Q4Metrics.yhat - Q4_actual.acTot
Q4Metrics

In [ ]:
filename = 'Y:\Team7\Working\FBP_2020Q4_total.csv'
Q4Metrics.to_csv(filename)

# Modeling for 2021 Q1 blind submission 

In [ ]:
#1All data
df = edvi_BY_DAY[['acTot']]
df = df.reset_index()
df = df.rename(columns={"record_create_time": "ds", "acTot": "y"})
df['ds']= to_datetime(df['ds'])
df['covidRegressor']= covidRegressor['regressor']

In [ ]:
#1
model = Prophet(changepoint_range=0.8, changepoint_prior_scale=0.25, daily_seasonality=True, seasonality_mode='multiplicative')
model.add_country_holidays(country_name='US')
model.add_regressor('covidRegressor')
model.fit(df)

In [ ]:
#1 Make future dates
future_data = model.make_future_dataframe(freq='d', periods=90)
future_data['covidRegressor']= covidRegressor['regressor']
future_data.tail()

In [ ]:
#1 Make forecast
forecast = model.predict(future_data)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
#1
fig1 = model.plot(forecast)

In [ ]:
#1 Trend changepoints
figure = model.plot(forecast)
for changepoint in model.changepoints:
    plt.axvline(changepoint,ls='--', lw=1)

In [ ]:
#1 Store forecast
Q1_forecast = forecast[['ds', 'yhat']].set_index('ds')
Q1_forecast['total'] = round(Q1_forecast['yhat'])#1 Make forecast

In [ ]:
#2All data
df = edvi_BY_DAY[['lowAcuity']]
df = df.reset_index()
df = df.rename(columns={"record_create_time": "ds", "lowAcuity": "y"})
df['ds']= to_datetime(df['ds'])
df['covidRegressor']= covidRegressor['regressor']

In [ ]:
#2
model = Prophet(changepoint_range=0.8, changepoint_prior_scale=0.25, daily_seasonality=True, seasonality_mode='multiplicative')
model.add_country_holidays(country_name='US')
model.add_regressor('covidRegressor')
model.fit(df)

In [ ]:
#2 Make future dates
future_data = model.make_future_dataframe(freq='d', periods=90)
future_data['covidRegressor']= covidRegressor['regressor']
future_data.tail()

In [ ]:
#2 Make forecast
forecast = model.predict(future_data)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
#2 Store forecast
Q1_forecast2 = forecast[['ds', 'yhat']].set_index('ds')
Q1_forecast2['lowAcuity'] = round(Q1_forecast2['yhat'])

In [ ]:
Q1_forecast2

In [ ]:
#3All data
df = edvi_BY_DAY[['highAcuity']]
df = df.reset_index()
df = df.rename(columns={"record_create_time": "ds", "highAcuity": "y"})
df['ds']= to_datetime(df['ds'])
df['covidRegressor']= covidRegressor['regressor']

In [ ]:
#3
model = Prophet(changepoint_range=0.8, changepoint_prior_scale=0.25, daily_seasonality=True, seasonality_mode='multiplicative')
model.add_country_holidays(country_name='US')
model.add_regressor('covidRegressor')
model.fit(df)

In [ ]:
#3 Make future dates
future_data = model.make_future_dataframe(freq='d', periods=90)
future_data['covidRegressor']= covidRegressor['regressor']
future_data.tail()

In [ ]:
#3 Make forecast
forecast = model.predict(future_data)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
#3 Store forecast
Q1_forecast3 = forecast[['ds', 'yhat']].set_index('ds')
Q1_forecast3['highAcuity'] = round(Q1_forecast3['yhat'])

In [ ]:
Q1_forecast = Q1_forecast.loc['2021':'2021']
Q1_forecast2 = Q1_forecast2.loc['2021':'2021']
Q1_forecast3 = Q1_forecast3.loc['2021':'2021']

result1 = pd.merge(Q1_forecast, Q1_forecast3, on="ds")
output = pd.merge(result1, Q1_forecast2, on="ds")

output.to_csv('Y:\Team7\Working\Team17_prophet_final.csv')